In [ ]:
pip install pandas numpy scikit-learn tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

Load Datase

In [ ]:
file_path = '/content/Datasets/banana_quality_dataset.csv'
data = pd.read_csv(file_path)

Target and Feature

In [ ]:
target = 'quality_category'
features = ['quality_score', 'sugar_content_brix', 'firmness_kgf',
            'length_cm', 'weight_g', 'tree_age_years', 'altitude_m',
            'rainfall_mm', 'soil_nitrogen_ppm']

Encoding Target

In [ ]:
# Encode the target
label_encoder = LabelEncoder()
data[target] = label_encoder.fit_transform(data[target])

Split data into features (X) and target (y)

In [ ]:
X = data[features]
y = to_categorical(data[target])

Standardization

In [ ]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Split X and Y to Train and Test Data

In [ ]:
# Split into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 20% validation

Build and Compile Model

In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Train Model

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3460 - loss: 1.3275 - val_accuracy: 0.6850 - val_loss: 0.9797
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6727 - loss: 0.9128 - val_accuracy: 0.7900 - val_loss: 0.8004
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7664 - loss: 0.7326 - val_accuracy: 0.8300 - val_loss: 0.7135
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8254 - loss: 0.6080 - val_accuracy: 0.8450 - val_loss: 0.6450
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8301 - loss: 0.5682 - val_accuracy: 0.8600 - val_loss: 0.5862
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8652 - loss: 0.5119 - val_accuracy: 0.8650 - val_loss: 0.5338
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8524 - loss: 0.5217 - val_accuracy: 0.8800 - val_loss: 0.4766
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8908 - loss: 0.4221 - val_accuracy: 0.8900 - val_loss

Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9316 - loss: 0.1556 
Test Accuracy: 93.50%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Make predictions
y_pred_prob = model.predict(X_test)  # Model predictions (probabilities)

# Convert predictions to class labels
y_pred = np.argmax(y_pred_prob, axis=1)  # Predicted class indices
y_true = np.argmax(y_test, axis=1)       # True class indices (one-hot encoded to labels)

# Generate classification report
report = classification_report(y_true, y_pred, target_names=label_encoder.classes_)
print("Classification Report:")
print(report)

# Confusion matrix (optional)
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Classification Report:
              precision    recall  f1-score   support

        Good       0.92      0.98      0.95        86
     Premium       1.00      0.25      0.40         4
  Processing       0.94      0.96      0.95       101
      Unripe       1.00      0.56      0.71         9

    accuracy                           0.94       200
   macro avg       0.97      0.69      0.75       200
weighted avg       0.94      0.94      0.93       200

Confusion Matrix:
[[84  0  2  0]
 [ 3  1  0  0]
 [ 4  0 97  0]
 [ 0  0  4  5]]
